<a href="https://colab.research.google.com/github/john-d-noble/callcenter/blob/main/Daily_Forecast_JSON_File_Parameters.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import xgboost as xgb
import yfinance as yf
from datetime import date, timedelta

# --- Configuration ---
SAVED_MODEL_FILE = 'final_xgboost_model.json'
TICKERS = ['BTC-USD', 'ETH-USD', 'SOL-USD', '^VIX']

print("🚀 Loading the final model to make a new prediction...")

try:
    # 1. Get Fresh Market Data for the Prediction
    # In a real scenario, you'd get today's closing data to predict tomorrow.
    # We will get yesterday's data as a stand-in for the "latest" data.
    yesterday = date.today() - timedelta(days=1)

    print(f"--- Fetching latest market data for {yesterday.strftime('%Y-%m-%d')} ---")
    latest_data = yf.download(TICKERS, start=yesterday, period='1d')

    # 2. Prepare the Data for the Model
    # The model needs the data in the exact same format it was trained on.
    prices = latest_data['Close'].rename(columns={
        'BTC-USD': 'btc_price', 'ETH-USD': 'eth_price',
        'SOL-USD': 'sol_price', '^VIX': 'vix_price'})

    volumes = latest_data['Volume'].rename(columns={
        'BTC-USD': 'btc_volume', 'ETH-USD': 'eth_volume',
        'SOL-USD': 'sol_volume', '^VIX': 'vix_volume'})

    # Combine into a single row of data
    prediction_input = pd.concat([prices, volumes], axis=1)

    # Ensure the column order is identical to the training data
    training_columns = ['btc_price', 'eth_price', 'sol_price', 'vix_price',
                        'btc_volume', 'eth_volume', 'sol_volume', 'vix_volume']
    prediction_input = prediction_input[training_columns]


    # 3. Load the Saved Model and Make a Prediction
    final_model = xgb.XGBRegressor()
    final_model.load_model(SAVED_MODEL_FILE)

    prediction = final_model.predict(prediction_input)
    predicted_volume = int(prediction[0])

    print("\n--- Prediction Complete ---")
    print(f"✅ Based on the latest market data, the model forecasts a call volume of: {predicted_volume:,}")


except Exception as e:
    print(f"❌ An error occurred: {e}")

🚀 Loading the final model to make a new prediction...
--- Fetching latest market data for 2025-08-23 ---


/tmp/ipython-input-3502916663.py:19: FutureWarning: YF.download() has changed argument auto_adjust default to True
  latest_data = yf.download(TICKERS, start=yesterday, period='1d')
[*********************100%***********************]  4 of 4 completed



--- Prediction Complete ---
✅ Based on the latest market data, the model forecasts a call volume of: 395,468
